In [1]:
%matplotlib qt5
import sys
sys.path.insert(1, '/home/harsh/CourseworkRepo/rh/rhv2src/python/')
from os.path import getsize
from xdr_tools import *
from _vacuumtoair import *
import h5py
import matplotlib.pyplot as plt

In [2]:
__XDR_Specs__ = dict()
__XDR_Specs__['atmos_3d_input'] = XDR_Specs(
    [
        ['Nx', int, None],
        ['Ny', int, None],
        ['Nz', int, None],
        ['NHydr', int, None],
        ['Top', int, None],
        ['Bot', int, None],
        ['dx', float, None],
        ['dy', float, None],
        ['z', float, 'Nz'],
        ['T', float, '(Nx,Ny,Nz)'],
        ['ne', float, '(Nx,Ny,Nz)'],
        ['vturb', float, '(Nx,Ny,Nz)'],
        ['vx', float, '(Nx,Ny,Nz)'],
        ['vy', float, '(Nx,Ny,Nz)'],
        ['vz', float, '(Nx,Ny,Nz)'],
        ['nH', float, '(Nx,Ny,Nz,NHydr)']
    ]
)

In [3]:
specs = XDR_Specs(__XDR_Specs__['atmos_3d_input'])

In [4]:
reader = XDR_Reader(specs)
wt = XDR_Struct(specs)

In [5]:
f = h5py.File('/home/harsh/BifrostRun/bifrost_en024048_hion_0_504_0_504_-500000.0_3000000.0.nc', 'r')

In [6]:
f.keys()

<KeysViewHDF5 ['B_x', 'B_y', 'B_z', 'depth', 'electron_density', 'hydrogen_populations', 'nhydr', 'snapshot_number', 'temperature', 'velocity_x', 'velocity_y', 'velocity_z', 'x', 'y', 'z']>

In [7]:
height = f['z'][0, 0, 0]/1e3
ind = np.argmin(np.abs(height - 2000))
print(height[ind])
plt.imshow(f['temperature'][0, :, :, ind], cmap='gray', origin='lower')

2000.552875


In [8]:
f['velocity_x'][0, 200, 200]/1e3

array([ 0.80144086,  0.766992  ,  0.74393109,  0.73156201,  0.71660303,
        0.70923309,  0.72862616,  0.76409045,  0.78255255,  0.8241875 ,
        0.8538089 ,  0.91681415,  1.00760785,  1.11441125,  1.25221423,
        1.37750842,  1.50729272,  1.57101917,  1.65087134,  1.70153857,
        1.73374756,  1.72640308,  1.74263037,  1.75936804,  1.79209839,
        1.77750525,  1.8237832 ,  1.89799585,  2.02307947,  2.08800171,
        1.95015259,  1.95231897,  2.12087036,  2.16536035,  2.1814458 ,
        2.26200635,  2.43976392,  2.79066187,  3.18836548,  3.37090991,
        3.51282031,  3.67958521,  3.78777368,  3.92629419,  4.00840283,
        3.97025684,  3.94000464,  3.89164185,  3.87704712,  3.80220557,
        3.7505918 ,  3.56468896,  3.34897681,  3.09533398,  2.64917017,
        2.09251392,  1.48177319,  0.98913196,  0.52487445,  0.0601281 ,
       -0.27326648, -0.57763489, -0.83331317, -0.96296484, -1.04695972,
       -1.0871571 , -1.11266687, -1.05135852, -0.85537427, -0.49

In [10]:
NX1 = 280
NX2 = 311
NY1 = 210
NY2 = 241
plt.imshow(f['temperature'][0, NX1:NX2, NY1:NY2, ind], cmap='gray', origin='lower')

In [11]:
wt['Nx'] = NX2-NX1

In [12]:
wt['Ny'] = NY2-NY1

In [13]:
wt['Nz'] = f['z'].shape[-1]

In [14]:
wt['NHydr'] = f['nhydr'][()].size

In [15]:
f.attrs.keys()

<KeysViewHDF5 ['comment', 'boundary_top', 'boundary_bottom', 'has_B', 'description', 'nx', 'ny', 'nz', 'nt', '_NCProperties']>

In [16]:
f.attrs['description']

b'Bifrost Simulation (simulation_name, start_x, end_x, start_y, end_y, height_min_in_m, height_max_in_m) - en024048_hion - 0 - 504 - 0 - 504 - -500000.0 - 3000000.0'

In [17]:
f.attrs['boundary_top'][()][0], f.attrs['boundary_bottom'][()][0]

(0, 1)

In [18]:
wt['Top'] = 1

In [19]:
wt['Bot'] = 2

In [20]:
wt['dx'] = 48.0

In [21]:
wt['dy'] = 48.0

In [22]:
wt['z'] = f['z'][0, 0, 1]/1e3

In [23]:
f['temperature'].shape

(1, 504, 504, 177)

In [24]:
wt['T'] = f['temperature'][0, NX1:NX2, NY1:NY2]

In [25]:
wt['ne'] = f['electron_density'][0, NX1:NX2, NY1:NY2]

In [26]:
wt['vturb'] = np.zeros((NX2-NX1, NY2-NY1, f['z'].shape[-1]))

In [27]:
wt['vx'] = f['velocity_x'][0, NX1:NX2, NY1:NY2]/1e3

In [28]:
wt['vy'] = f['velocity_y'][0, NX1:NX2, NY1:NY2]/1e3

In [29]:
wt['vz'] = f['velocity_z'][0, NX1:NX2, NY1:NY2] / 1e3

In [30]:
wt['nH'] = np.transpose(f['hydrogen_populations'][0, :, NX1:NX2, NY1:NY2], axes=(1, 2, 3, 0))

In [31]:
wt

Nx           <class 'int'>
Ny           <class 'int'>
Nz           <class 'int'>
NHydr        <class 'int'>
Top          <class 'int'>
Bot          <class 'int'>
dx           <class 'float'>
dy           <class 'float'>
z            <class 'numpy.ndarray'>         (177,)
T            <class 'numpy.ndarray'>         (31, 31, 177)
ne           <class 'numpy.ndarray'>         (31, 31, 177)
vturb        <class 'numpy.ndarray'>         (31, 31, 177)
vx           <class 'numpy.ndarray'>         (31, 31, 177)
vy           <class 'numpy.ndarray'>         (31, 31, 177)
vz           <class 'numpy.ndarray'>         (31, 31, 177)
nH           <class 'numpy.ndarray'>         (31, 31, 177, 6)

In [32]:
reader.write('/home/harsh/BifrostRun/bifrost_en024048_hion_{}_{}_{}_{}_-500000.0_3000000.0.dat'.format(NX1, NX2, NY1, NY2), wt)

In [33]:
__XDR_Specs__['b_3d'] = XDR_Specs(
    [
        ['B', float, '(Nx,Ny,Nz)'],
        ['Binc', float, '(Nx,Ny,Nz)'],
        ['Bazi', float, '(Nx,Ny,Nz)'],
    ]
)

In [34]:
Babs = np.sqrt(
        np.add(
            np.square(f['B_x'][0, NX1:NX2, NY1:NY2]),
            np.add(
                np.square(f['B_y'][0, NX1:NX2, NY1:NY2]),
                np.square(f['B_z'][0, NX1:NX2, NY1:NY2])
            )
        )
    )

In [35]:
Babs.shape

(31, 31, 177)

In [36]:
Binc = np.arccos(np.divide(f['B_z'][0, NX1:NX2, NY1:NY2], Babs))

In [37]:
Binc.shape

(31, 31, 177)

In [38]:
Bazi = np.arctan2(f['B_y'][0, NX1:NX2, NY1:NY2], f['B_x'][0, NX1:NX2, NY1:NY2])

In [39]:
Bazi.shape

(31, 31, 177)

In [40]:
specs = XDR_Specs(__XDR_Specs__['b_3d'])
reader = XDR_Reader(specs)
wt = XDR_Struct(specs)

In [41]:
wt['B'] = Babs
wt['Binc'] = Binc
wt['Bazi'] = Bazi

In [42]:
wt

B            <class 'numpy.ndarray'>         (31, 31, 177)
Binc         <class 'numpy.ndarray'>         (31, 31, 177)
Bazi         <class 'numpy.ndarray'>         (31, 31, 177)

In [43]:
reader.write('/home/harsh/BifrostRun/bifrost_en024048_hion_{}_{}_{}_{}_-500000.0_3000000.0_B.dat'.format(NX1, NX2, NY1, NY2), wt)